# Group 8 Assessment 2 Part 2 - Wrangling the Data

This Notebook outlines the steps to wrangle the datasets for Group 8's project. The following data is wrangeld:

1. Flight Delay dataset
2. Monthly aviation fuel price data
3. Coordinates of Australian airports (longitude and latitude)
4. Coordinates of Bureau of Meteorology(BoM) weather stations (longitude and latitude)
5. Long term monthly average weather statistics from these BoM stations

The final output is a parquet file with all the data wrangled and joined together.

## Preparatory code - import libraries, start spark session, define functions

In [ ]:
//import libraries
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.spark.sql.{DataFrame}
import scala.io.Source
import sys.process._
import java.net.URL
import java.io.File
import org.apache.spark.sql.Row
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import scala.math._
import org.apache.spark.ml.feature.Imputer


In [2]:
// Start a simple Spark Session
val spark = SparkSession.builder().getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1e739cd2


In [3]:
//Scala equivalent of R melt data frame function
//Acquired from https://stackoverflow.com/questions/41670103/how-to-melt-spark-dataframe

import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame}

/** Extends the [[org.apache.spark.sql.DataFrame]] class
 *
 *  @param df the data frame to melt
 */
implicit class DataFrameFunctions(df: DataFrame) {

    /** Convert [[org.apache.spark.sql.DataFrame]] from wide to long format.
     * 
     *  melt is (kind of) the inverse of pivot
     *  melt is currently (02/2017) not implemented in spark
     *
     *  @see reshape packe in R (https://cran.r-project.org/web/packages/reshape/index.html)
     *  @see this is a scala adaptation of http://stackoverflow.com/questions/41670103/pandas-melt-function-in-apache-spark
     *  
     *  @todo method overloading for simple calling
     *
     *  @param id_vars the columns to preserve
     *  @param value_vars the columns to melt
     *  @param var_name the name for the column holding the melted columns names
     *  @param value_name the name for the column holding the values of the melted columns
     *
     */

    def melt(
            id_vars: Seq[String], value_vars: Seq[String], 
            var_name: String = "variable", value_name: String = "value") : DataFrame = {

        // Create array<struct<variable: str, value: ...>>
        val _vars_and_vals = array((for (c <- value_vars) yield { struct(lit(c).alias(var_name), col(c).alias(value_name)) }): _*)

        // Add to the DataFrame and explode
        val _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

        val cols = id_vars.map(col _) ++ { for (x <- List(var_name, value_name)) yield { col("_vars_and_vals")(x).alias(x) }}

        return _tmp.select(cols: _*)

    }
}

import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
defined class DataFrameFunctions


## Read in Monthly Flight Delay Data

In [4]:
//flight delay data obtained from
//https://data.gov.au/data/dataset/29128ebd-dbaa-4ff5-8b86-d9f30de56452/resource/cf663ed1-0c5e-497f-aea9-e74bfda9cf44/download/otp_time_series_web.csv
val flightDelay_csv = "otp_time_series_web.csv"

//Define Flight Delay Schema
val flightDelaySchema = StructType(List(
    StructField("Route", StringType, nullable = true),
    StructField("Departing_Port", StringType, nullable = true),
    StructField("Arriving_Port", StringType, nullable = true),
    StructField("Airline", StringType, nullable = true),
    StructField("Month", StringType, nullable = true),
    StructField("Sectors_Scheduled", IntegerType, nullable = true),
    StructField("Sectors_Flown", IntegerType, nullable = true),
    StructField("Cancellations", IntegerType, nullable = true),
    StructField("Departures_On_Time", IntegerType, nullable = true),
    StructField("Arrivals_On_Time", IntegerType, nullable = true),
    StructField("Departures_Delayed", IntegerType, nullable = true),
    StructField("Arrivals_Delayed", IntegerType, nullable = true),
    StructField("Year", IntegerType, nullable = true),
    StructField("Month_Num", StringType, nullable = true)
))

//read in Flight Delay data to data frame
val readflightDelayDF = spark.read.schema(flightDelaySchema)
  .option("delimiter",",")
  .option("header", "true")
  .csv(flightDelay_csv)
  .where(($"Airline" =!= "All Airlines") )   //Remove All Ports from monthly totals
  .where($"Route" =!= "All Ports-All Ports") //Remove All Airlines from monthly totals
  .select("Month",
          "Departing_Port", 
          "Arriving_Port", 
          "Airline", 
          "Sectors_Flown",
          "Departures_Delayed",
          "Year", 
          "Month_Num")

readflightDelayDF.show()
readflightDelayDF.getClass

+------+--------------+-------------+-------+-------------+------------------+----+---------+
| Month|Departing_Port|Arriving_Port|Airline|Sectors_Flown|Departures_Delayed|Year|Month_Num|
+------+--------------+-------------+-------+-------------+------------------+----+---------+
|Jan-04|      Adelaide|     Brisbane| Qantas|           93|                12|2004|        1|
|Jan-04|      Adelaide|     Canberra| Qantas|           48|                 3|2004|        1|
|Jan-04|      Adelaide|   Gold Coast| Qantas|            9|                 1|2004|        1|
|Jan-04|      Adelaide|    Melbourne| Qantas|          350|                33|2004|        1|
|Jan-04|      Adelaide|        Perth| Qantas|          130|                 9|2004|        1|
|Jan-04|      Adelaide|       Sydney| Qantas|          331|                18|2004|        1|
|Jan-04| Alice Springs|       Sydney| Qantas|           33|                 2|2004|        1|
|Jan-04|      Brisbane|     Adelaide| Qantas|           93| 

flightDelay_csv: String = otp_time_series_web.csv
flightDelaySchema: org.apache.spark.sql.types.StructType = StructType(StructField(Route,StringType,true), StructField(Departing_Port,StringType,true), StructField(Arriving_Port,StringType,true), StructField(Airline,StringType,true), StructField(Month,StringType,true), StructField(Sectors_Scheduled,IntegerType,true), StructField(Sectors_Flown,IntegerType,true), StructField(Cancellations,IntegerType,true), StructField(Departures_On_Time,IntegerType,true), StructField(Arrivals_On_Time,IntegerType,true), StructField(Departures_Delayed,IntegerType,true), StructField(Arrivals_Delayed,IntegerType,true), StructField(Year,IntegerType,true), StructField(Month_Num,StringType,true))
readflightDelayDF: org.apache.spark.sql.DataFrame = [Month: string,...

## Read in Monthly Fuel Price data

In [5]:
// Fuel price data obtained from:
// https://www.indexmundi.com/commodities/?commodity=jet-fuel&months=240&currency=aud

val jetFuel_csv = "jetFuel.csv"

//Read in data
val fuelDF = spark.read
  .option("delimiter",",")
  .option("header", "true")
  .option("inferSchema", "true")
  .csv(jetFuel_csv)

fuelDF.show()

+------+-----+-------+
| Month|Price| Change|
+------+-----+-------+
|Apr-00|  1.2|      -|
|May-00| 1.32|  9.58%|
|Jun-00| 1.32|  0.34%|
|Jul-00| 1.35|  2.29%|
|Aug-00| 1.55| 14.43%|
|Sep-00| 1.84| 18.71%|
|Oct-00| 1.86|  1.33%|
|Nov-00| 1.97|  5.76%|
|Dec-00| 1.58|-19.65%|
|Jan-01| 1.56| -1.11%|
|Feb-01| 1.52| -2.59%|
|Mar-01| 1.49| -2.50%|
|Apr-01| 1.54|  3.66%|
|May-01| 1.58|  2.53%|
|Jun-01| 1.48| -6.19%|
|Jul-01| 1.39| -5.89%|
|Aug-01| 1.46|  4.41%|
|Sep-01| 1.46|  0.24%|
|Oct-01| 1.23|-15.63%|
|Nov-01| 1.05|-14.71%|
+------+-----+-------+
only showing top 20 rows



jetFuel_csv: String = jetFuel.csv
fuelDF: org.apache.spark.sql.DataFrame = [Month: string, Price: double ... 1 more field]


## Read in Australian Airport longitudes and latitudes

In [6]:
// Airport coordinates obtained from:
//"https://datahub.io/core/airport-codes/r/airport-codes.csv"
val airport_csv = "airport-codes.csv"

// Read in Airport coordinate data
val airport = spark.read
  .option("delimiter",",")
  .option("header", "true")
  .option("inferSchema", "true")
  .csv(airport_csv) 
  .where($"iso_country" === "AU") //only want Aussie airports
  .where($"type" === "large_airport" || $"type" === "medium_airport") //only types large airport and medium
   //split lat and lon into two columns
  .withColumn("lon", split($"coordinates", ",")(0))
  .withColumn("lat", split($"coordinates", ",")(1))
   // only want name, iso_region, lon, lat
  .select("name","iso_region","lon","lat")

//clean up airport names to align with flightDelay airport names
val airport_cleaned = airport
                        .withColumn("lon",col("lon").cast(DoubleType))
                        .withColumn("lat",col("lat").cast(DoubleType))
                        .withColumn("name", regexp_replace(col("name"), " Airport", ""))
                        .withColumn("name", regexp_replace(col("name"), " International", ""))
                        .withColumn("name", regexp_replace(col("name"), " Boulder", ""))
                        .withColumn("name", regexp_replace(col("name"), " Kingsford Smith", ""))
                        .withColumn("name", regexp_replace(col("name"), " Connellan", ""))
                        .withColumn("name", regexp_replace(col("name"), " City", ""))
                        .withColumn("name", regexp_replace(col("name"), " Whitsunday Coast", ""))
                        .withColumn("name", regexp_replace(col("name"), " Byron Gateway", ""))
                        .withColumn("name", regexp_replace(col("name"), " Regional", ""))
                        .withColumn("name", regexp_replace(col("name"), "Wynyard", "Burnie"))

airport_cleaned.show()


+------------------+----------+------------------+-------------------+
|              name|iso_region|               lon|                lat|
+------------------+----------+------------------+-------------------+
|   (Duplicate)YKCY|    AU-QLD|               0.1|                0.1|
|            Albany|     AU-WA|117.80899810791016| -34.94329833984375|
|RAAF Base Amberley|    AU-QLD|        152.712006|           -27.6406|
|             Alpha|    AU-QLD|           146.584|         -23.646099|
|            Ararat|    AU-VIC|142.98899841308594| -37.30939865112305|
|          Armidale|    AU-NSW|     151.617004395|-30.528099060099997|
|        Ayers Rock|     AU-NT|        130.975998|           -25.1861|
|        Barcaldine|    AU-QLD|     145.307006836|     -23.5652999878|
|     Alice Springs|     AU-NT|133.90199279785156|-23.806699752807617|
|          Brisbane|    AU-QLD|153.11700439453125|-27.384199142456055|
|        Gold Coast|    AU-QLD|     153.505004883|     -28.1644001007|
|     

airport_csv: String = airport-codes.csv
airport: org.apache.spark.sql.DataFrame = [name: string, iso_region: string ... 2 more fields]
airport_cleaned: org.apache.spark.sql.DataFrame = [name: string, iso_region: string ... 2 more fields]


## Read in BoM weather station longitude and latitudes

Most BoM weather stations measure rainfall. Therefore the rainfall BoM station directory is a good proxy list for every single BoM weather station.

In [7]:
// BoM weather station coordinates obtained from:
// http://www.bom.gov.au/climate/data/lists_by_element/alphaAUS_136.txt

val rain_txt = "alphaAUS_136.txt"

//read in bom coordinates to rdd
val rain_rdd = sc.textFile(rain_txt)

//fixed width file - align columns
val rain = rain_rdd.map(l => (
                        l.substring(0, 8).trim(), 
                        l.substring(8, 48).trim(),
                        l.substring(48, 57).trim(),
                        l.substring(59, 67).trim(),
                        l.substring(67,71).trim(),
                        l.substring(71,76).trim(),
                        l.substring(77,80).trim(),
                        l.substring(80,86).trim(),
                        l.substring(85, 93).trim(),
                        l.substring(94, 98).trim(),
                        l.substring(98,101).trim()
                        ))
             .toDF("site","bom_name","lat","lon","start_month","start_year","end_month","end_year","years","%","AWS")

val rain_cleaned = rain.where(($"start_year" <= 2004))   // bom site operating since at least 2004, aligning with flightDelay
                       .where(($"end_year" >= 2019))   // bom site operating since at least 2019, aligning with flightDelay
                       //remove problem bom sites that turned out to have no monthly published dataset
                       .where($"site" =!= 34015)
                       .where($"site" =!= 23055)
                       .where($"site" =!= 59009)
                       .where($"site" =!= 34100)
                       .where($"site" =!= 34078)
                       .where($"site" =!= 23065)
                       .where($"site" =!= 23006)
                       .withColumn("lon",col("lon").cast(DoubleType))
                       .withColumn("lat",col("lat").cast(DoubleType))
                       .select("site","bom_name","lon","lat")
                 
rain_cleaned.show()
                       

+-----+--------------------+--------+--------+
| site|            bom_name|     lon|     lat|
+-----+--------------------+--------+--------+
|67114|ABBOTSBURY (FAIRF...|150.8611|-33.8667|
|66034|ABBOTSFORD (BLACK...|151.1295|-33.8507|
|39000|            ABERCORN|151.1292|-25.1333|
|39319|         ABERCORN TM|151.1306|-25.1292|
|91313|ABERDEEN (MELROSE...|146.3339|-41.2478|
|61065| ABERDEEN (ROSSGOLE)|150.7285|-32.1402|
|32174|    ABERGOWRIE ALERT|145.9364|-18.4889|
|30000|ABINGDON DOWNS ST...|143.1792|-17.6083|
|88119|ACHERON RIVER @ T...|145.7167|-37.3167|
| 9971|          ACTON PARK|115.4072|-33.7845|
|71040|   ADAMINABY (YAOUK)|148.8106|-35.7858|
|71000|ADAMINABY ALPINE ...|148.7693|-35.9962|
|89106|           ADDINGTON|143.6718|-37.4152|
|23001| ADELAIDE (BRIGHTON)|  138.52|-35.0199|
|23093|ADELAIDE (CLARENC...| 138.579|-34.9726|
|23005|ADELAIDE (GLEN OS...|138.6519|-34.9464|
|23096|ADELAIDE (HOPE VA...|138.6844|-34.8564|
|23090|ADELAIDE (KENT TOWN)|138.6216|-34.9211|
|23115|  ADEL

rain_txt: String = alphaAUS_136.txt
rain_rdd: org.apache.spark.rdd.RDD[String] = alphaAUS_136.txt MapPartitionsRDD[33] at textFile at <console>:61
rain: org.apache.spark.sql.DataFrame = [site: string, bom_name: string ... 9 more fields]
rain_cleaned: org.apache.spark.sql.DataFrame = [site: string, bom_name: string ... 2 more fields]


## Find nearest BoM weather station to each airport

Using the Haversine calculation for distances between coordinates, find the best source of BoM data for each airport

In [8]:
// Using Haversine formula calculate distances between airports and Bom stations, and determine which is closest
// adapted from https://dzone.com/articles/scala-calculating-distance-between-two-locations

//Lists of bom site, lon and lat data to search. these use i as the indice in the code
var a: List[Double] = rain_cleaned.select("lon").collect().map(_(0).asInstanceOf[Double]).toList
var b: List[Double] = rain_cleaned.select("lat").collect().map(_(0).asInstanceOf[Double]).toList
var z: List[String] = rain_cleaned.select("site").collect().map(_(0).asInstanceOf[String]).toList
var rain_length = rain_cleaned.count().asInstanceOf[Int]

//lists of airports, lon and lat data to search. these use j as the indice in the code
var c: List[Double] = airport_cleaned.select("lon").collect().map(_(0).asInstanceOf[Double]).toList
var d: List[Double] = airport_cleaned.select("lat").collect().map(_(0).asInstanceOf[Double]).toList
var y: List[String] = airport_cleaned.select("name").collect().map(_(0).asInstanceOf[String]).toList
var airport_length = airport_cleaned.count().asInstanceOf[Int]

// lists of best matches for closest airport => bom station. Starts with none found and a large distance away
var best_station_id: Array[String] = Array.fill(airport_cleaned.count().asInstanceOf[Int])("none found yet")
var best_distance: Array[Double] = Array.fill(airport_cleaned.count().asInstanceOf[Int])(999999)

//Runs through all combinations of airport + weather station, calculates haversine distance, and if the locations are closer 
//any previously calculated, updated best_station_id and best_distance with the new best combination
for (i <- 0 to (rain_length-1)){
    for (j <- 0 to (airport_length-1)){
        val lonDistance = Math.toRadians(a(i) - c(j))
        val latDistance = Math.toRadians(b(i) - d(j))
        val sinLon = Math.sin(lonDistance / 2)
        val sinLat = Math.sin(latDistance / 2)
        val e = sinLat * sinLat +
            (Math.cos(Math.toRadians(b(i))) * Math.cos(Math.toRadians(d(j))) *sinLon * sinLon)
        val f = Math.atan2(Math.sqrt(e), Math.sqrt(1 - e))*2*6371 //Earth radius in km
        //if a better match is found, update best_station_id and best_distance
        if (best_distance(j) > f){
            best_distance(j) = f
            best_station_id(j) = z(i)
        }        
    }   
}

//Produce data frame with best bom_station for each airport
var best_station_id_df= sc.parallelize(best_station_id).toDF("best_station_id").withColumn("rowID",monotonicallyIncreasingId)
var y_df= sc.parallelize(y).toDF("name").withColumn("rowID",monotonicallyIncreasingId)
val airport_cleaned_with_best = y_df.join(best_station_id_df, Seq("rowid"))
                                    .select("name","best_station_id")
val airport_cleaned_with_best_ID_name = airport_cleaned_with_best
                                            .join(rain_cleaned, 
                                                  airport_cleaned_with_best("best_station_id") 
                                                  === rain_cleaned("site"), "left")
                                            .select("name","best_station_id","bom_name")

airport_cleaned_with_best_ID_name.show()


+------------------+---------------+--------------------+
|              name|best_station_id|            bom_name|
+------------------+---------------+--------------------+
|      Port Hedland|           4032|PORT HEDLAND AIRPORT|
|       Rockhampton|          39083|    ROCKHAMPTON AERO|
|        Shepparton|          81125|  SHEPPARTON AIRPORT|
|            Ceduna|          18012|          CEDUNA AMO|
|      Shellharbour|          68241|ALBION PARK (SHEL...|
|            Bourke|          48245|  BOURKE AIRPORT AWS|
|            Telfer|          13030|         TELFER AERO|
|            Mackay|          33045|         MACKAY AERO|
|          Corryong|          82171|TINTALDRA (WARRAR...|
|RAAF Base Richmond|          67105|       RICHMOND RAAF|
|          Armidale|          56238|ARMIDALE AIRPORT AWS|
|           Renmark|          24048|        RENMARK AERO|
|        Glen Innes|          56243|GLEN INNES AIRPOR...|
|             Tumut|          72044|TUMUT (SIMPSON ST...|
|         Glad

a: List[Double] = List(150.8611, 151.1295, 151.1292, 151.1306, 146.3339, 150.7285, 145.9364, 143.1792, 145.7167, 115.4072, 148.8106, 148.7693, 143.6718, 138.52, 138.579, 138.6519, 138.6844, 138.6216, 138.5827, 138.5274, 138.5425, 138.6125, 138.6434, 138.7051, 138.5832, 138.5196, 131.1042, 147.9419, 148.0641, 122.2167, 149.1424, 153.4242, 144.0883, 148.41, 140.5036, 140.5003, 117.8808, 150.79, 150.7761, 146.5694, 147.3194, 146.9531, 146.9509, 136.8391, 153.0026, 139.6615, 116.3336, 138.7419, 148.2811, 148.2288, 148.2447, 117.2219, 145.7116, 115.1892, 133.889, 148.7697, 134.4028, 145.4408, 119.0958, 146.2969, 146.8343, 144.6797, 139.2828, 142.49, 147.2467, 146.6406, 144.8262, 147.4833, 134.6841, 152.6989, 152.7111, 144.4033, 118.2214, 137.7697, 137.7217, 117.0736, 135.2382, 118.6319, 147....

## Join The Monthly Fuel Price to Flight Delay Data

In [9]:
// Join flightDelay to fuel
val flightDelayDF = readflightDelayDF.join(fuelDF, readflightDelayDF("Month") === fuelDF("Month"), "left")
flightDelayDF.count()

//Explode # of flights delayed to of row per flight as a binary value 1 == flight delayed, 0 == flight on time
val delays = flightDelayDF.withColumn("Departures_Delayed",
                            explode(array_repeat(lit(1), $"Departures_Delayed")))
val ontimes = flightDelayDF.withColumn("Departures_Delayed",
                            explode(array_repeat(lit(0), $"Sectors_Flown"-$"Departures_Delayed")))

//Recreate data frame with # of flight delayed replicated in each row
val flightDelayReplicatedDF = delays.union(ontimes).drop("Sectors_Flown", "Month")

flightDelayReplicatedDF.orderBy(col("Year").asc,col("Month_Num").asc).show(1000)
flightDelayReplicatedDF.printSchema()
flightDelayReplicatedDF.count()

+--------------+-------------+-------+------------------+----+---------+-----+------+
|Departing_Port|Arriving_Port|Airline|Departures_Delayed|Year|Month_Num|Price|Change|
+--------------+-------------+-------+------------------+----+---------+-----+------+
|        Sydney|       Darwin| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|       Darwin| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|        Perth| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|         Perth|     Brisbane| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|       Darwin| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Cairns| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|        Perth| Qantas|                

|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Brisbane| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Brisbane| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Brisbane| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Brisbane| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Brisbane| Qantas|                

|      Brisbane|       Cairns| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Cairns| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Cairns| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Cairns| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Cairns| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Cairns| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Cairns| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Cairns| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|     Canberra| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|     Canberra| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|     Canberra| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Darwin| Qantas|                

|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Kalgoorlie|        Perth| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                

|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|Alice Springs| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Brisbane| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Brisbane| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Brisbane| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Brisbane| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Brisbane| Qantas|                

...

|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|      Brisbane|       Sydney| Qantas|                

|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Gold Coast|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Kalgoorlie|        Perth| Qantas|                 1|2004|        1|  1.3| 9.32%|
|    Kalgoorlie|        Perth| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|     Adelaide| Qantas|                

|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                 1|2004|        1|  1.3| 9.32%|
|     Melbourne|       Sydney| Qantas|                

|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|     Adelaide| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|Alice Springs| Qantas|                 1|2004|        1|  1.3| 9.32%|
|        Sydney|Alice Springs| Qantas|                

flightDelayDF: org.apache.spark.sql.DataFrame = [Month: string, Departing_Port: string ... 9 more fields]
delays: org.apache.spark.sql.DataFrame = [Month: string, Departing_Port: string ... 9 more fields]
ontimes: org.apache.spark.sql.DataFrame = [Month: string, Departing_Port: string ... 9 more fields]
flightDelayReplicatedDF: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 6 more fields]
res5: Long = 6296897


## Join BoM station ID and name to Flight Delay Data

The will be accomplished by two joins, one for Departing Port and one for Arriving Port


In [10]:
//Join Flight Delay to Departing Port BoM station ID
val flightDelayRep_Depart = flightDelayReplicatedDF
                                .join(airport_cleaned_with_best_ID_name, 
                                      flightDelayReplicatedDF("Departing_Port") 
                                      === airport_cleaned_with_best_ID_name("name"), "left")
                                    .withColumnRenamed("best_station_id","Departing_Port_station_ID")
                                    .withColumnRenamed("bom_name","Departing_Port_station_name")
                                    .drop("name")

//Join Flight Delay,Departing Port BoM station ID, Arriving Port BoM station ID together
val flightDelayRep_Depart_Arrive = flightDelayRep_Depart
                                        .join(airport_cleaned_with_best_ID_name, 
                                          flightDelayRep_Depart("Arriving_Port") 
                                          === airport_cleaned_with_best_ID_name("name"), "left")
                                        .withColumnRenamed("best_station_id","Arriving_Port_station_ID")
                                        .withColumnRenamed("bom_name","Arriving_Port_station_name")
                                        .drop("name")
    
flightDelayRep_Depart_Arrive.show()

+--------------+-------------+----------+------------------+----+---------+-----+------+-------------------------+---------------------------+------------------------+--------------------------+
|Departing_Port|Arriving_Port|   Airline|Departures_Delayed|Year|Month_Num|Price|Change|Departing_Port_station_ID|Departing_Port_station_name|Arriving_Port_station_ID|Arriving_Port_station_name|
+--------------+-------------+----------+------------------+----+---------+-----+------+-------------------------+---------------------------+------------------------+--------------------------+
|      Adelaide| Port Lincoln|QantasLink|                 0|2018|        1| 2.46| 3.39%|                    23024|           ADELAIDE AIRPORT|                   18192|      NORTH SHIELDS (PO...|
|      Adelaide| Port Lincoln|QantasLink|                 0|2018|        1| 2.46| 3.39%|                    23024|           ADELAIDE AIRPORT|                   18192|      NORTH SHIELDS (PO...|
|      Adelaide| Port Lin

flightDelayRep_Depart: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 8 more fields]
flightDelayRep_Depart_Arrive: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 10 more fields]


## Wrangle the BoM monthly summary statistics 

For the 44 weather stations selected to be used with the Flight Delay data, corresponding monthly average statistics files were retrieved from the BoM website. A list of corresponding URLs are in the file 45stations.csv (The 45th station is all the airports, and was discarded from the FlightDelay data earlier.)

In [11]:
//Read in the stations data
val stations = spark.read.format("csv").option("header", "true").load("./station_files/*.csv")
val stations_df = stations.drop("Annual","Number.of.Years","Start.Year","End.Year")
                          .withColumnRenamed("Statistic.Element","Statistic")
                            //Select Mean value attributes only
                          .where(
$"Statistic" ===  "Mean maximum temperature Degrees C"||
$"Statistic" ===  "Mean number of days >= 30 Degrees C"||
$"Statistic" ===  "Mean number of days >= 35 Degrees C"||
$"Statistic" ===  "Mean number of days >= 40 Degrees C"||
$"Statistic" ===  "Mean minimum temperature Degrees C"||
$"Statistic" ===  "Mean number of days <= 2 Degrees C"||
$"Statistic" ===  "Mean number of days <= 0 Degrees C"||
$"Statistic" ===  "Mean daily ground minimum temperature Degrees C"||
$"Statistic" ===  "Mean rainfall mm"||
$"Statistic" ===  "Mean number of days of rain"||
$"Statistic" ===  "Mean number of days of rain >= 1 mm"||
$"Statistic" ===  "Mean number of days of rain >= 10 mm"||
$"Statistic" ===  "Mean number of days of rain >= 25 mm"||
$"Statistic" ===  "Mean daily wind run km"||
$"Statistic" ===  "Mean daily sunshine hours"||
$"Statistic" ===  "Mean daily solar exposure MJ/m*m"||
$"Statistic" ===  "Mean number of clear days"||
$"Statistic" ===  "Mean number of cloudy days"||
$"Statistic" ===  "Mean daily evaporation mm"||
$"Statistic" ===  "Mean 9am temperature Degrees C"||
$"Statistic" ===  "Mean 9am wet bulb temperature Degrees C"||
$"Statistic" ===  "Mean 9am dew point temperature Degrees C"||
$"Statistic" ===  "Mean 9am relative humidity %"||
$"Statistic" ===  "Mean 9am cloud cover okas"||
$"Statistic" ===  "Mean 9am wind speed km/h"||
$"Statistic" ===  "Mean 3pm temperature Degrees C"||
$"Statistic" ===  "Mean 3pm wet bulb temperature Degrees C"||
$"Statistic" ===  "Mean 3pm dew point temperature Degrees C"||
$"Statistic" ===  "Mean 3pm relative humidity %"||
$"Statistic" ===  "Mean 3pm cloud cover oktas"||
$"Statistic" ===  "Mean 3pm wind speed km/h")
                      
//Melt data frame
val id_vars: Seq[String] = Seq("station_id","Statistic")
val value_vars: Seq[String] = Seq("January","February","March","April","May","June","July","August"
                                          ,"September","October","November","December")
val var_name: String = "month"
val value_name: String = "value"
val stations_df_melted = stations_df.melt(id_vars,value_vars,var_name,value_name)
                                    .withColumn("double",col("value").cast("Double"))
                                    .drop("value")
                                    //change month names to month number - align with flightDelay
                                    .withColumn("month", regexp_replace(col("month"), "January", "1"))
                                    .withColumn("month", regexp_replace(col("month"), "February", "2"))
                                    .withColumn("month", regexp_replace(col("month"), "March", "3"))
                                    .withColumn("month", regexp_replace(col("month"), "April", "4"))
                                    .withColumn("month", regexp_replace(col("month"), "May", "5"))
                                    .withColumn("month", regexp_replace(col("month"), "June", "6"))
                                    .withColumn("month", regexp_replace(col("month"), "July", "7"))
                                    .withColumn("month", regexp_replace(col("month"), "August", "8"))
                                    .withColumn("month", regexp_replace(col("month"), "September", "9"))
                                    .withColumn("month", regexp_replace(col("month"), "October", "10"))
                                    .withColumn("month", regexp_replace(col("month"), "November", "11"))
                                    .withColumn("month", regexp_replace(col("month"), "December", "12"))

//Recast melted data frame 
val stations_df_pivoted = stations_df_melted.groupBy("station_id","month").pivot("Statistic").sum("double")

stations_df_pivoted.collect.foreach(println)


2020-05-24 03:32:36,434 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
[3003,2,5.1,24.2,67.0,31.6,26.5,18.7,5.7,24.3,74.0,29.8,26.0,12.9,7.2,25.1,22.5,7.9,363.0,32.9,26.1,3.5,15.2,0.0,0.0,26.9,3.3,0.2,11.3,9.1,4.6,2.0,179.6]
[39326,6,null,10.6,50.0,22.3,16.1,16.9,null,11.1,67.0,17.7,14.4,12.4,null,null,13.6,null,230.0,23.5,13.3,null,null,0.0,0.0,0.0,0.0,0.0,6.2,3.6,1.0,0.3,38.6]
[34038,6,3.2,8.8,44.0,22.9,15.6,6.6,2.8,10.2,73.0,15.4,12.8,5.5,3.5,null,14.6,null,null,23.7,11.2,13.4,6.6,0.0,0.1,0.2,0.0,0.0,3.2,2.1,0.6,0.3,22.1]
[56238,11,null,9.3,54.0,20.0,14.2,19.0,null,10.2,70.0,16.0,12.9,18.3,null,null,23.4,null,351.0,22.9,10.1,null,null,0.1,0.2,1.7,0.0,0.0,12.5,8.5,3.3,0.9,95.6]
[59006,11,null,null,null,null,null,null,null,null,null,null,null,null,null,null,22.0,null,null,27.1,14.8,null,null,null,null,null,null,n

[91107,9,4.9,6.4,68.0,12.8,9.9,22.6,4.9,6.8,79.0,10.4,8.7,16.1,null,null,13.1,null,394.0,14.5,4.9,2.9,13.5,1.9,6.9,0.0,0.0,0.0,18.1,10.7,2.6,0.4,95.7]
[35264,1,4.8,17.0,41.0,33.1,22.9,15.4,4.6,19.2,63.0,27.3,22.0,15.0,null,null,24.9,null,311.0,34.6,22.3,4.7,8.6,0.0,0.0,28.7,14.9,1.1,7.8,5.8,2.5,1.0,84.2]
[66037,10,4.4,9.9,54.0,20.7,15.0,24.6,4.4,10.2,61.0,18.4,14.1,16.3,6.0,11.1,20.1,7.9,479.0,22.7,13.3,8.2,11.1,0.0,0.0,2.3,0.4,0.0,10.6,7.8,2.0,0.6,70.1]
[65070,6,null,6.6,57.0,15.4,11.2,17.1,null,6.5,86.0,8.6,7.7,13.6,null,null,9.5,null,291.0,16.4,4.4,null,null,3.9,9.1,0.0,0.0,0.0,10.2,5.5,1.6,0.5,48.9]
[15635,8,null,-0.4,25.0,22.3,12.3,16.7,null,1.2,48.0,12.5,7.9,13.5,null,null,18.4,null,299.0,23.7,5.9,null,null,0.9,4.6,2.2,0.0,0.0,1.3,1.0,0.1,0.0,4.8]
[29127,5,2.8,6.0,29.0,27.0,16.1,12.8,2.4,7.1,46.0,19.8,13.5,11.4,6.4,11.1,17.9,9.4,298.0,28.0,13.9,16.0,5.2,0.0,0.0,9.1,0.2,0.0,1.8,1.2,0.4,0.1,11.9]
[59006,10,null,null,null,null,null,null,null,null,null,null,null,null,null,null,20.8,n

[15590,10,2.6,1.0,19.0,30.0,15.9,16.0,2.2,2.1,30.0,23.3,13.4,16.9,10.3,11.2,25.2,10.1,290.0,31.2,14.8,17.8,4.3,0.0,0.0,18.8,8.1,0.3,4.4,2.7,0.6,0.1,20.0]
[33045,10,2.6,17.9,60.0,26.5,21.2,22.2,3.5,17.6,63.0,25.5,20.7,16.8,null,null,24.7,null,444.0,28.4,18.2,12.2,3.8,0.0,0.0,6.0,0.1,0.0,6.0,3.7,1.1,0.2,34.5]
[72150,8,4.8,5.3,59.0,13.7,9.8,14.3,4.6,4.9,83.0,7.8,6.5,8.7,1.9,0.7,11.5,6.4,247.0,14.5,3.5,7.4,14.2,5.3,11.0,0.0,0.0,0.0,12.9,8.7,1.2,0.1,50.7]
[23024,2,3.2,11.0,42.0,26.3,17.8,22.0,3.5,11.0,53.0,21.7,15.8,11.7,8.4,12.4,24.3,10.0,392.0,28.1,16.2,12.1,6.5,0.0,0.0,9.5,3.5,0.4,3.8,2.3,0.5,0.2,18.8]
[61078,11,4.4,13.1,55.0,23.8,17.8,23.5,4.6,13.4,66.0,20.5,16.5,14.4,6.3,11.7,22.1,7.6,378.0,25.6,14.4,6.7,11.2,0.0,0.0,5.5,1.6,0.2,11.6,7.2,2.3,0.5,81.9]
[12038,10,3.1,2.3,27.0,24.7,14.1,17.6,3.0,5.5,47.0,17.9,12.0,17.1,8.4,9.4,23.8,null,375.0,26.0,11.2,13.9,5.9,0.0,0.1,7.1,1.5,0.1,4.4,2.7,0.4,0.1,15.8]
[23024,12,3.8,9.4,45.0,23.6,16.0,23.1,4.6,9.3,53.0,20.2,14.5,15.6,8.3,11.2,26.9,9.4,424

[40861,6,null,12.3,63.0,20.1,15.9,18.2,null,12.3,76.0,16.8,14.5,15.0,null,null,11.9,null,341.0,21.4,11.4,null,null,0.0,0.1,0.0,0.0,0.0,16.2,9.4,3.4,1.2,120.5]
[23024,9,4.6,7.2,56.0,16.7,12.0,21.2,4.8,7.1,66.0,14.1,10.8,17.8,4.0,5.5,16.1,7.2,410.0,18.2,9.0,5.9,12.1,0.0,0.1,0.2,0.0,0.0,12.9,8.5,1.1,0.1,45.0]
[33255,12,3.8,23.0,73.0,28.6,24.8,23.7,4.1,22.9,76.0,27.7,24.4,19.8,null,null,24.4,null,null,30.0,24.4,8.1,7.2,0.0,0.0,16.1,0.1,0.0,10.8,8.3,3.8,2.0,184.9]
[39326,2,null,20.7,62.0,29.0,23.5,22.7,null,20.7,69.0,27.0,22.9,15.9,null,null,22.5,null,348.0,30.7,23.0,null,null,0.0,0.0,19.9,0.6,0.0,12.0,8.5,3.5,1.9,176.0]
[15590,2,3.8,8.0,25.0,34.1,19.7,16.5,2.9,10.1,39.0,27.6,18.0,12.1,11.9,17.6,25.5,9.8,324.0,35.2,20.8,12.1,6.5,0.0,0.0,25.3,16.7,2.6,4.5,3.4,1.1,0.4,41.2]
[56238,9,null,3.4,46.0,16.1,9.9,21.1,null,4.9,64.0,12.1,8.4,19.9,null,null,18.7,null,373.0,17.7,4.7,null,null,2.3,7.0,0.0,0.0,0.0,9.8,5.4,1.6,0.5,50.2]
[33045,4,4.4,20.0,69.0,26.5,22.3,21.8,4.3,20.2,79.0,24.3,21.7,15.7,nul

[66037,11,4.6,12.3,56.0,22.1,16.7,25.3,4.7,12.4,64.0,19.9,15.8,16.0,6.6,14.0,21.7,7.7,517.0,24.2,15.5,6.4,11.9,0.0,0.0,3.2,1.0,0.1,11.2,8.3,2.3,0.6,79.9]
[60139,4,null,14.9,64.0,22.5,18.1,17.3,null,14.9,76.0,19.5,16.9,12.3,null,null,15.0,null,null,24.4,13.9,null,null,0.0,0.0,0.2,0.0,0.0,13.6,9.1,3.7,1.6,131.3]
[40717,1,null,20.1,69.0,26.6,22.4,22.9,null,19.8,70.0,25.7,22.1,18.1,null,null,24.2,null,364.0,28.5,21.0,null,null,0.0,0.0,5.0,0.0,0.0,14.3,10.3,4.0,1.6,160.2]
[76031,5,4.7,7.1,50.0,18.3,12.7,15.1,4.0,7.6,81.0,10.8,9.3,9.5,2.7,4.6,10.7,6.6,256.0,19.1,7.4,8.0,11.7,0.2,1.5,0.0,0.0,0.0,6.8,4.1,0.7,0.1,25.3]
[39128,10,3.2,14.6,54.0,25.1,19.0,20.6,4.1,15.1,61.0,23.2,18.5,15.2,6.1,null,22.7,null,358.0,27.1,16.5,9.0,7.2,0.0,0.0,2.5,0.0,0.0,8.2,5.0,1.7,0.6,77.4]
[8050,10,2.9,13.4,61.0,21.4,16.8,17.7,3.5,11.5,60.0,20.0,15.4,13.7,null,null,24.4,null,null,23.0,12.6,8.4,3.9,0.0,0.0,1.9,0.2,0.0,6.1,2.7,0.2,0.0,17.5]
[33247,8,3.1,12.5,51.0,24.1,17.8,21.4,3.3,13.6,68.0,20.2,16.7,12.3,null,null,

...

stations: org.apache.spark.sql.DataFrame = [Statistic.Element: string, January: string ... 16 more fields]
stations_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Statistic: string, January: string ... 12 more fields]
id_vars: Seq[String] = List(station_id, Statistic)
value_vars: Seq[String] = List(January, February, March, April, May, June, July, August, September, October, November, December)
var_name: String = month
value_name: String = value
stations_df_melted: org.apache.spark.sql.DataFrame = [station_id: string, Statistic: string ... 2 more fields]
stations_df_pivoted: org.apache.spark.sql.DataFrame = [station_id: string, month: string ... 31 more fields]



[29127,6,2.1,3.5,28.0,24.1,14.1,13.1,1.9,4.9,50.0,15.9,10.8,10.2,5.3,7.2,16.5,9.5,302.0,24.9,10.0,19.8,3.2,0.1,0.6,1.7,0.0,0.0,1.1,0.7,0.2,0.1,6.4]
[18192,10,null,8.7,56.0,18.3,13.3,25.5,null,8.8,64.0,16.3,12.5,21.3,null,null,20.3,null,412.0,21.0,9.6,null,null,0.0,0.0,2.5,0.6,0.0,9.0,4.3,0.6,0.1,24.1]
[40861,2,null,21.1,71.0,27.1,23.2,23.5,null,20.8,73.0,26.3,22.8,18.8,null,null,21.6,null,456.0,28.9,21.3,null,null,0.0,0.0,6.3,0.3,0.0,15.2,11.3,4.6,2.3,207.5]
[65070,4,null,7.0,37.0,23.7,15.0,16.8,null,9.8,64.0,17.0,13.2,16.8,null,null,15.7,null,303.0,24.9,10.4,null,null,0.2,0.4,2.1,0.0,0.0,4.2,3.0,1.0,0.5,31.4]
[15635,3,null,6.0,22.0,33.3,18.7,16.6,null,7.6,36.0,25.3,16.3,19.9,null,null,23.0,null,357.0,34.7,19.4,null,null,0.0,0.0,25.9,15.8,2.4,2.8,1.8,0.7,0.4,30.8]
[70072,11,null,null,null,null,null,null,3.2,null,62.0,16.9,13.0,null,null,null,23.6,null,null,24.4,8.9,null,null,null,null,null,null,null,6.9,6.1,1.9,0.5,58.9]
[55325,10,null,6.8,38.0,23.8,15.1,17.5,null,9.2,56.0,18.8,13.8,1

[33045,7,3.1,13.6,63.0,21.3,17.0,20.5,3.3,13.0,76.0,17.5,15.1,12.2,null,null,16.0,null,340.0,22.7,11.4,12.8,5.9,0.0,0.1,0.0,0.0,0.0,7.7,3.6,0.9,0.3,33.1]
[23024,4,4.4,9.1,50.0,20.8,14.7,18.8,4.6,9.2,61.0,17.4,13.2,13.0,4.3,7.9,13.7,7.3,330.0,22.3,11.9,7.9,12.0,0.0,0.0,1.7,0.1,0.0,8.4,5.7,1.0,0.2,34.7]
[66037,3,4.3,15.5,61.0,23.9,18.9,21.0,4.6,15.8,73.0,21.1,18.0,12.9,5.4,15.7,17.1,6.9,459.0,25.4,17.6,7.7,12.0,0.0,0.0,2.7,0.4,0.0,12.6,9.4,3.3,1.2,118.1]
[18192,3,null,12.0,54.0,22.4,16.8,24.1,null,12.6,69.0,18.8,15.6,17.0,null,null,18.2,null,421.0,24.4,14.6,null,null,0.0,0.0,3.5,1.1,0.3,5.8,2.4,0.5,0.2,17.4]
[72150,5,4.5,7.1,56.0,16.6,12.0,11.6,4.3,7.0,82.0,10.0,8.6,7.2,2.1,3.1,10.2,6.3,228.0,17.4,5.9,8.8,12.5,2.0,5.6,0.0,0.0,0.0,9.2,6.3,1.4,0.3,50.6]
[32040,8,2.6,13.3,52.0,24.6,18.2,21.9,3.0,13.1,63.0,21.1,16.8,10.0,6.1,12.4,19.5,9.1,329.0,26.1,14.7,16.4,4.7,0.0,0.0,0.5,0.0,0.0,2.6,1.4,0.4,0.1,15.5]
[91311,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,16.4,null

[14015,11,4.7,22.4,58.0,31.9,25.5,17.7,5.0,23.4,72.0,29.2,25.2,8.7,7.3,25.0,22.9,8.4,301.0,33.3,25.3,3.6,11.5,0.0,0.0,29.3,2.6,0.0,12.4,10.0,4.3,1.7,142.7]
[32040,11,3.0,19.7,58.0,29.2,23.1,23.1,4.4,19.6,63.0,27.7,22.6,15.1,9.1,21.7,25.7,9.5,438.0,30.8,22.9,9.1,5.9,0.0,0.0,22.0,0.7,0.1,7.2,4.8,1.5,0.6,57.1]
[40861,4,null,17.6,68.0,24.3,20.2,21.2,null,17.5,75.0,22.5,19.5,17.3,null,null,16.6,null,392.0,26.1,17.1,null,null,0.0,0.0,0.8,0.0,0.0,16.2,10.9,4.4,1.7,148.8]
[94008,8,5.4,4.3,60.0,12.4,8.7,17.6,4.8,3.9,76.0,8.2,6.4,13.8,2.0,1.7,8.5,5.7,378.0,13.5,4.7,3.8,14.5,0.6,4.8,0.0,0.0,0.0,13.9,8.9,1.0,0.2,46.6]
[39083,3,4.8,18.4,54.0,29.4,22.4,16.2,4.3,19.7,72.0,25.3,21.7,14.2,6.2,19.1,20.7,null,314.0,30.6,20.9,6.2,10.9,0.0,0.0,18.9,1.2,0.0,10.2,7.2,2.6,1.2,104.8]
[72146,1,3.8,10.0,33.0,29.2,18.5,9.8,3.2,12.8,63.0,20.5,16.2,4.3,null,null,27.0,null,null,31.2,15.5,11.8,8.0,0.0,0.0,18.8,6.8,1.1,6.2,4.6,1.5,0.4,47.9]
[32040,3,4.7,21.4,63.0,29.5,24.2,19.6,4.6,21.2,71.0,27.2,23.4,10.2,7.0,21.6,21

[12038,9,3.0,2.0,31.0,21.3,12.5,17.8,2.9,4.9,54.0,14.8,10.3,16.2,5.8,5.8,19.1,null,341.0,22.4,8.1,14.1,6.2,0.1,0.8,1.7,0.1,0.0,5.4,2.9,0.3,0.0,13.7]
[72150,12,3.9,6.5,30.0,27.8,16.8,17.7,3.0,10.0,52.0,21.0,15.1,13.0,9.4,10.8,27.2,10.4,334.0,29.6,14.0,12.7,8.0,0.0,0.0,14.7,4.4,0.5,6.3,4.6,1.5,0.5,46.1]
[9021,2,2.4,12.3,37.0,30.2,19.9,20.3,2.7,12.7,53.0,23.5,17.4,17.8,9.6,15.6,25.6,11.0,473.0,32.0,17.5,16.0,4.8,0.0,0.0,18.5,7.7,1.1,2.8,1.5,0.4,0.1,15.0]
[70072,3,null,null,null,null,null,null,3.3,null,70.0,16.6,13.6,null,null,null,18.2,null,null,25.6,10.7,null,null,null,null,null,null,null,5.1,4.3,1.7,0.5,51.2]
[70072,12,null,null,null,null,null,null,3.3,null,61.0,19.2,14.9,null,null,null,25.8,null,null,27.6,11.4,null,null,null,null,null,null,null,6.1,5.3,1.9,0.5,56.0]
[91107,5,4.6,8.1,70.0,13.8,11.1,17.7,4.8,7.7,86.0,9.9,8.8,11.7,null,null,6.8,null,342.0,15.2,5.4,4.4,11.7,1.9,6.4,0.0,0.0,0.0,17.3,9.7,2.6,0.4,98.2]
[55325,6,null,5.6,52.0,16.0,11.0,14.2,null,6.4,83.0,9.4,8.0,9.1,null,null,


## Impute null values in BoM monthly summary statistics

The BoM statistics contain null values. These null values will be imputed as the mean of the attribute. The mean is selected so that when models are built in Part 3 the attribute will have a mean effect on the model output.

In [12]:
//Array of attributes that will be imputed
val forImpute: Array[String] = Array(
"Mean maximum temperature Degrees C",
"Mean number of days >= 30 Degrees C",
"Mean number of days >= 35 Degrees C",
"Mean number of days >= 40 Degrees C",
"Mean minimum temperature Degrees C",
"Mean number of days <= 2 Degrees C",
"Mean number of days <= 0 Degrees C",
"Mean daily ground minimum temperature Degrees C",
"Mean rainfall mm",
"Mean number of days of rain",
"Mean number of days of rain >= 1 mm",
"Mean number of days of rain >= 10 mm",
"Mean number of days of rain >= 25 mm",
"Mean daily wind run km",
"Mean daily sunshine hours",
"Mean daily solar exposure MJ/m*m",
"Mean number of clear days",
"Mean number of cloudy days",
"Mean daily evaporation mm",
"Mean 9am temperature Degrees C",
"Mean 9am wet bulb temperature Degrees C",
"Mean 9am dew point temperature Degrees C",
"Mean 9am relative humidity %",
"Mean 9am cloud cover okas",
"Mean 9am wind speed km/h",
"Mean 3pm temperature Degrees C",
"Mean 3pm wet bulb temperature Degrees C",
"Mean 3pm dew point temperature Degrees C",
"Mean 3pm relative humidity %",
"Mean 3pm cloud cover oktas",
"Mean 3pm wind speed km/h")

//Impute means for null entries
val imputer = new Imputer()
  .setInputCols(forImpute)
  .setOutputCols(forImpute)
  .setStrategy("mean")
val stations_df_imputed = imputer.fit(stations_df_pivoted)
                  .transform(stations_df_pivoted)

stations_df_imputed.printSchema
stations_df_imputed.collect.foreach(println)


root
 |-- station_id: string (nullable = true)
 |-- month: string (nullable = false)
 |-- Mean 3pm cloud cover oktas: double (nullable = true)
 |-- Mean 3pm dew point temperature Degrees C: double (nullable = true)
 |-- Mean 3pm relative humidity %: double (nullable = true)
 |-- Mean 3pm temperature Degrees C: double (nullable = true)
 |-- Mean 3pm wet bulb temperature Degrees C: double (nullable = true)
 |-- Mean 3pm wind speed km/h: double (nullable = true)
 |-- Mean 9am cloud cover okas: double (nullable = true)
 |-- Mean 9am dew point temperature Degrees C: double (nullable = true)
 |-- Mean 9am relative humidity %: double (nullable = true)
 |-- Mean 9am temperature Degrees C: double (nullable = true)
 |-- Mean 9am wet bulb temperature Degrees C: double (nullable = true)
 |-- Mean 9am wind speed km/h: double (nullable = true)
 |-- Mean daily evaporation mm: double (nullable = true)
 |-- Mean daily ground minimum temperature Degrees C: double (nullable = true)
 |-- Mean daily solar 

[60139,8,3.9157738095238095,7.8,52.0,18.7,13.3,19.0,3.8741379310344843,7.8,68.0,14.0,11.1,12.6,6.180833333333333,12.575925925925922,14.5,8.12388888888889,347.72072072072075,20.3,6.6,9.595535714285718,8.847321428571426,0.8,2.9,0.1,0.0,0.0,7.9,5.3,1.8,0.6,61.7]
[39128,3,4.2,18.5,59.0,27.7,21.9,20.7,4.1,19.4,71.0,25.1,21.5,15.7,6.0,12.575925925925922,20.1,8.12388888888889,368.0,29.3,20.1,6.8,8.5,0.0,0.0,10.3,0.1,0.0,11.8,6.8,2.2,0.8,111.4]
[14015,2,6.5,24.2,72.0,30.0,25.9,18.6,6.5,24.5,83.0,27.7,25.5,11.1,5.7,24.7,19.4,5.9,350.0,31.5,24.8,0.7,21.6,0.0,0.0,23.2,0.1,0.0,20.2,17.8,9.9,4.8,370.0]
[59006,8,3.9157738095238095,11.76524390243902,50.670731707317074,24.30528455284553,17.44857723577236,18.819512195121945,3.8741379310344843,12.310162601626015,65.14087301587301,19.84623015873015,15.694047619047616,14.408333333333333,6.180833333333333,12.575925925925922,15.1,8.12388888888889,347.72072072072075,20.4,5.3,9.595535714285718,8.847321428571426,0.4436507936507935,1.1373015873015875,8.35337301

[39128,4,4.1,16.2,57.0,25.9,20.0,19.0,3.2,17.1,70.0,22.9,19.4,15.8,4.6,12.575925925925922,17.7,8.12388888888889,342.0,27.5,17.6,10.4,6.4,0.0,0.0,2.3,0.0,0.0,8.9,4.8,1.1,0.4,55.8]
[15635,4,3.9157738095238095,4.4,24.0,29.0,16.2,15.0,3.8741379310344843,5.9,42.0,20.4,13.1,17.3,6.180833333333333,12.575925925925922,19.3,8.12388888888889,318.0,30.1,14.6,9.595535714285718,8.847321428571426,0.0,0.0,15.4,3.8,0.0,2.1,1.6,0.4,0.1,14.7]
[76031,4,3.7,7.2,40.0,22.7,14.7,15.4,3.1,8.6,68.0,14.9,11.8,11.6,4.6,7.1,15.0,8.3,266.0,23.7,10.2,12.1,8.4,0.0,0.0,3.0,0.2,0.0,4.3,2.8,0.5,0.1,19.5]
[40717,6,3.9157738095238095,11.2,60.0,19.7,15.3,16.8,3.8741379310344843,11.5,71.0,17.0,14.2,13.4,6.180833333333333,12.575925925925922,11.4,8.12388888888889,261.0,21.0,11.4,9.595535714285718,8.847321428571426,0.0,0.1,0.0,0.0,0.0,13.2,8.3,3.1,1.7,139.1]
[40717,7,3.9157738095238095,9.7,56.0,19.4,14.4,18.3,3.8741379310344843,10.0,67.0,16.4,13.1,13.5,6.180833333333333,12.575925925925922,12.8,8.12388888888889,256.0,20.7,10.1,

[33247,10,3.2,16.5,51.0,28.3,21.1,23.1,4.6,17.5,61.0,25.8,20.7,16.9,6.180833333333333,12.575925925925922,24.3,8.12388888888889,339.0,30.3,17.4,4.5,3.9,0.0,0.0,16.1,1.2,0.0,6.2,3.6,0.9,0.3,36.8]
[15590,10,2.6,1.0,19.0,30.0,15.9,16.0,2.2,2.1,30.0,23.3,13.4,16.9,10.3,11.2,25.2,10.1,290.0,31.2,14.8,17.8,4.3,0.0,0.0,18.8,8.1,0.3,4.4,2.7,0.6,0.1,20.0]
[33045,10,2.6,17.9,60.0,26.5,21.2,22.2,3.5,17.6,63.0,25.5,20.7,16.8,6.180833333333333,12.575925925925922,24.7,8.12388888888889,444.0,28.4,18.2,12.2,3.8,0.0,0.0,6.0,0.1,0.0,6.0,3.7,1.1,0.2,34.5]
[72150,8,4.8,5.3,59.0,13.7,9.8,14.3,4.6,4.9,83.0,7.8,6.5,8.7,1.9,0.7,11.5,6.4,247.0,14.5,3.5,7.4,14.2,5.3,11.0,0.0,0.0,0.0,12.9,8.7,1.2,0.1,50.7]
[23024,2,3.2,11.0,42.0,26.3,17.8,22.0,3.5,11.0,53.0,21.7,15.8,11.7,8.4,12.4,24.3,10.0,392.0,28.1,16.2,12.1,6.5,0.0,0.0,9.5,3.5,0.4,3.8,2.3,0.5,0.2,18.8]
[61078,11,4.4,13.1,55.0,23.8,17.8,23.5,4.6,13.4,66.0,20.5,16.5,14.4,6.3,11.7,22.1,7.6,378.0,25.6,14.4,6.7,11.2,0.0,0.0,5.5,1.6,0.2,11.6,7.2,2.3,0.5,81.9]
[1203

[15590,8,1.7,-0.6,25.0,21.8,11.8,14.8,1.7,0.8,46.0,13.4,8.0,8.2,5.7,2.2,18.9,9.9,244.0,22.7,5.9,22.5,3.3,2.0,6.8,2.3,0.0,0.0,1.8,1.2,0.2,0.1,8.6]
[72146,6,4.8,6.9,68.0,13.1,10.2,5.0,5.2,5.5,94.0,6.4,6.1,2.0,6.180833333333333,12.575925925925922,7.1,8.12388888888889,347.72072072072075,13.9,3.4,6.0,14.7,5.4,11.2,0.0,0.0,0.0,13.5,9.4,2.2,0.2,68.6]
[65070,12,3.9157738095238095,8.3,30.0,29.7,18.1,19.4,3.8741379310344843,11.7,52.0,22.8,16.7,19.9,6.180833333333333,12.575925925925922,27.3,8.12388888888889,375.0,31.6,16.0,9.595535714285718,8.847321428571426,0.0,0.0,19.7,7.5,0.7,6.6,4.8,2.0,0.7,58.9]
[39326,1,3.9157738095238095,20.4,60.0,29.2,23.4,24.1,3.8741379310344843,20.3,66.0,27.4,22.8,17.0,6.180833333333333,12.575925925925922,24.7,8.12388888888889,374.0,30.8,23.1,9.595535714285718,8.847321428571426,0.0,0.0,21.2,1.0,0.0,11.6,8.5,3.2,1.5,143.3]
[56238,3,3.9157738095238095,9.8,53.0,21.3,15.3,18.0,3.8741379310344843,12.0,79.0,16.0,13.5,16.2,6.180833333333333,12.575925925925922,19.0,8.1238888888

...

[40861,9,3.9157738095238095,14.2,63.0,22.3,17.5,23.5,3.8741379310344843,13.4,65.0,20.8,16.7,16.7,6.180833333333333,12.575925925925922,20.2,8.12388888888889,370.0,24.4,12.9,9.595535714285718,8.847321428571426,0.0,0.0,1.1,0.0,0.0,10.1,5.3,1.5,0.7,52.9]
[91311,9,3.9157738095238095,11.76524390243902,50.670731707317074,24.30528455284553,17.44857723577236,18.819512195121945,3.8741379310344843,12.310162601626015,65.14087301587301,19.84623015873015,15.694047619047616,14.408333333333333,6.180833333333333,12.575925925925922,13.2,8.12388888888889,385.0,15.2,4.0,9.595535714285718,8.847321428571426,3.1,8.6,0.0,0.0,0.0,13.9,8.5,1.8,0.1,55.6]
[91126,4,4.9,8.7,62.0,16.5,12.6,22.5,4.9,8.4,74.0,13.2,10.9,17.6,6.180833333333333,12.575925925925922,10.4,8.12388888888889,414.0,17.8,8.8,4.3,11.2,0.0,0.8,0.0,0.0,0.0,9.6,7.2,2.0,0.4,60.0]
[76031,9,4.3,4.2,40.0,19.4,12.2,19.7,3.4,6.3,67.0,12.7,9.7,15.7,4.6,4.1,17.1,8.2,331.0,20.6,7.3,9.7,9.2,0.1,1.0,1.1,0.1,0.0,7.1,4.3,0.8,0.1,26.6]
[34038,3,4.5,15.7,41.0,31.2,

[14015,11,4.7,22.4,58.0,31.9,25.5,17.7,5.0,23.4,72.0,29.2,25.2,8.7,7.3,25.0,22.9,8.4,301.0,33.3,25.3,3.6,11.5,0.0,0.0,29.3,2.6,0.0,12.4,10.0,4.3,1.7,142.7]
[32040,11,3.0,19.7,58.0,29.2,23.1,23.1,4.4,19.6,63.0,27.7,22.6,15.1,9.1,21.7,25.7,9.5,438.0,30.8,22.9,9.1,5.9,0.0,0.0,22.0,0.7,0.1,7.2,4.8,1.5,0.6,57.1]
[40861,4,3.9157738095238095,17.6,68.0,24.3,20.2,21.2,3.8741379310344843,17.5,75.0,22.5,19.5,17.3,6.180833333333333,12.575925925925922,16.6,8.12388888888889,392.0,26.1,17.1,9.595535714285718,8.847321428571426,0.0,0.0,0.8,0.0,0.0,16.2,10.9,4.4,1.7,148.8]
[94008,8,5.4,4.3,60.0,12.4,8.7,17.6,4.8,3.9,76.0,8.2,6.4,13.8,2.0,1.7,8.5,5.7,378.0,13.5,4.7,3.8,14.5,0.6,4.8,0.0,0.0,0.0,13.9,8.9,1.0,0.2,46.6]
[39083,3,4.8,18.4,54.0,29.4,22.4,16.2,4.3,19.7,72.0,25.3,21.7,14.2,6.2,19.1,20.7,8.12388888888889,314.0,30.6,20.9,6.2,10.9,0.0,0.0,18.9,1.2,0.0,10.2,7.2,2.6,1.2,104.8]
[72146,1,3.8,10.0,33.0,29.2,18.5,9.8,3.2,12.8,63.0,20.5,16.2,4.3,6.180833333333333,12.575925925925922,27.0,8.12388888888889,3

[23024,10,4.3,7.4,50.0,19.2,13.3,22.2,4.7,7.3,58.0,16.5,12.1,18.5,5.7,7.1,21.0,8.5,415.0,21.2,10.7,6.9,11.8,0.0,0.0,2.2,0.1,0.0,10.1,6.4,1.0,0.1,35.7]
[12038,6,4.3,4.9,48.0,16.8,11.2,15.7,3.8,6.0,74.0,11.0,8.7,11.8,2.6,4.0,10.4,8.12388888888889,269.0,17.6,6.3,9.1,10.4,0.7,3.8,0.0,0.0,0.0,8.5,4.8,0.7,0.1,27.3]
[61078,12,4.3,15.2,56.0,25.6,19.5,23.5,4.8,15.6,68.0,22.2,18.4,12.9,7.2,14.0,23.8,7.7,386.0,27.4,16.6,7.1,11.8,0.0,0.0,8.4,2.5,0.4,11.0,7.0,2.2,0.6,77.5]
[58198,3,3.9157738095238095,18.5,67.0,25.4,21.0,21.5,3.8741379310344843,18.6,80.0,22.5,20.1,12.5,6.180833333333333,12.575925925925922,18.3,8.12388888888889,270.0,27.1,18.4,9.595535714285718,8.847321428571426,0.0,0.0,2.5,0.1,0.0,18.1,14.2,5.5,2.5,201.9]
[61078,2,4.6,17.6,62.0,26.1,20.8,20.6,5.0,17.9,76.0,22.5,19.7,10.6,6.2,15.7,21.1,7.2,362.0,27.7,18.1,5.7,12.0,0.0,0.0,7.1,1.6,0.2,12.0,7.3,2.6,1.0,117.8]
[94008,10,5.3,5.7,54.0,15.6,10.9,22.5,5.2,5.5,64.0,12.4,9.3,16.4,4.2,4.5,17.3,7.4,429.0,17.4,7.6,3.1,14.0,0.1,0.6,0.1,0.0,0.0,13

[7176,10,3.9157738095238095,1.0,14.0,33.6,17.3,16.4,3.8741379310344843,3.5,24.0,27.2,15.4,16.7,6.180833333333333,12.575925925925922,26.3,8.12388888888889,278.0,35.3,17.9,9.595535714285718,8.847321428571426,0.0,0.0,28.3,18.7,1.4,1.6,1.1,0.2,0.0,5.8]
[31011,7,4.1,15.3,58.0,24.6,19.2,18.7,4.1,15.3,72.0,20.6,17.6,15.7,5.0,14.9,16.2,7.4,420.0,25.8,17.1,9.8,9.9,0.0,0.0,0.0,0.0,0.0,9.1,5.6,0.7,0.1,30.5]
[4032,3,3.2,19.7,45.0,34.7,25.1,21.6,3.1,18.6,51.0,31.4,23.5,15.1,9.3,23.6,23.5,8.12388888888889,402.0,36.8,24.6,14.0,6.7,0.0,0.0,30.2,22.4,5.4,4.6,3.0,1.2,0.6,55.3]
[29127,8,1.4,0.3,20.0,26.2,14.1,13.8,1.3,1.9,37.0,17.8,10.7,12.1,6.9,7.2,20.6,10.4,294.0,27.4,10.2,23.1,2.0,0.1,0.7,7.6,0.3,0.0,0.9,0.5,0.1,0.0,3.4]
[18192,8,3.9157738095238095,8.0,64.0,15.2,11.8,24.6,3.8741379310344843,8.0,77.0,12.2,10.2,18.1,6.180833333333333,12.575925925925922,11.9,8.12388888888889,386.0,16.8,7.1,9.595535714285718,8.847321428571426,0.0,0.1,0.0,0.0,0.0,17.8,9.5,1.2,0.0,50.8]
[40861,12,3.9157738095238095,19.9,69.

forImpute: Array[String] = Array(Mean maximum temperature Degrees C, Mean number of days >= 30 Degrees C, Mean number of days >= 35 Degrees C, Mean number of days >= 40 Degrees C, Mean minimum temperature Degrees C, Mean number of days <= 2 Degrees C, Mean number of days <= 0 Degrees C, Mean daily ground minimum temperature Degrees C, Mean rainfall mm, Mean number of days of rain, Mean number of days of rain >= 1 mm, Mean number of days of rain >= 10 mm, Mean number of days of rain >= 25 mm, Mean daily wind run km, Mean daily sunshine hours, Mean daily solar exposure MJ/m*m, Mean number of clear days, Mean number of cloudy days, Mean daily evaporation mm, Mean 9am temperature Degrees C, Mean 9am wet bulb temperature Degrees C, Mean 9am dew point temperature Degrees C, Mean 9am relative ...


## Join Bom Monthly summary statistics to Flight Delay Data

Finally, BoM monthly summary statistifcs will be joined to the Flight Delay Data - one set for Departing Port, and another for Arriving Port

In [13]:
//Rename station monthly data as two data frames, one for Arrive and Depart, for easing joining later
val stations_df_imputed_Depart = stations_df_imputed.toDF(stations_df_imputed.columns.map(c => s"${c}_Depart"): _*)
val stations_df_imputed_Arrive = stations_df_imputed.toDF(stations_df_imputed.columns.map(c => s"${c}_Arrive"): _*)

// join depart station monthly to flight delay
val flightDelayRep_Depart_Arrive_departBoM = flightDelayRep_Depart_Arrive
                                                .join(stations_df_imputed_Depart, 
                                                      flightDelayRep_Depart_Arrive("Departing_Port_station_ID") 
                                                      === stations_df_imputed_Depart("station_id_Depart") &&
                                                      flightDelayRep_Depart_Arrive("Month_Num") 
                                                      === stations_df_imputed_Depart("month_Depart")
                                                      , "left")
                                                .drop("station_id_Depart","month_Depart")

// join arrive station monthly to flight delay
val flightDelayRep_Depart_Arrive_departBoM_arriveBoM = flightDelayRep_Depart_Arrive_departBoM
                                                .join(stations_df_imputed_Arrive, 
                                                      flightDelayRep_Depart_Arrive_departBoM ("Arriving_Port_station_ID") 
                                                      === stations_df_imputed_Arrive("station_id_Arrive") &&
                                                      flightDelayRep_Depart_Arrive_departBoM ("Month_Num") 
                                                      === stations_df_imputed_Arrive("month_Arrive")
                                                      , "left")
                                                .drop("station_id_Arrive","month_Arrive")

flightDelayRep_Depart_Arrive_departBoM_arriveBoM.printSchema


root
 |-- Departing_Port: string (nullable = true)
 |-- Arriving_Port: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Departures_Delayed: integer (nullable = false)
 |-- Year: integer (nullable = true)
 |-- Month_Num: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Change: string (nullable = true)
 |-- Departing_Port_station_ID: string (nullable = true)
 |-- Departing_Port_station_name: string (nullable = true)
 |-- Arriving_Port_station_ID: string (nullable = true)
 |-- Arriving_Port_station_name: string (nullable = true)
 |-- Mean 3pm cloud cover oktas_Depart: double (nullable = true)
 |-- Mean 3pm dew point temperature Degrees C_Depart: double (nullable = true)
 |-- Mean 3pm relative humidity %_Depart: double (nullable = true)
 |-- Mean 3pm temperature Degrees C_Depart: double (nullable = true)
 |-- Mean 3pm wet bulb temperature Degrees C_Depart: double (nullable = true)
 |-- Mean 3pm wind speed km/h_Depart: double (nullable = true)
 |-- Mean

stations_df_imputed_Depart: org.apache.spark.sql.DataFrame = [station_id_Depart: string, month_Depart: string ... 31 more fields]
stations_df_imputed_Arrive: org.apache.spark.sql.DataFrame = [station_id_Arrive: string, month_Arrive: string ... 31 more fields]
flightDelayRep_Depart_Arrive_departBoM: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 41 more fields]
flightDelayRep_Depart_Arrive_departBoM_arriveBoM: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 72 more fields]


In [14]:
// Phew! now that the wrangling is done rename the data frame as something less hideous
// At the same remove special characters from attribute names to permit writing to Parquet file
val flightDelay = flightDelayRep_Depart_Arrive_departBoM_arriveBoM
                      .columns
                      .foldLeft(flightDelayRep_Depart_Arrive_departBoM_arriveBoM)
                            {(newdf, colname) 
                                 =>newdf.withColumnRenamed(colname, colname.replace(" ", "_").replace("=", ""))}


flightDelay: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 72 more fields]


## Write flightDelay in parquet format

In [15]:
//Write cleaned data frame
flightDelay.write.parquet("flightDelay.parquet")


2020-05-24 03:33:13,351 WARN  [Executor task launch worker for task 4022] hadoop.MemoryManager (MemoryManager.java:updateAllocation(115)) - Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
2020-05-24 03:33:13,354 WARN  [Executor task launch worker for task 4029] hadoop.MemoryManager (MemoryManager.java:updateAllocation(115)) - Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
2020-05-24 03:33:14,154 WARN  [Executor task launch worker for task 4028] hadoop.MemoryManager (MemoryManager.java:updateAllocation(115)) - Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
2020-05-24 03:33:14,513 WARN  [Executor task launch worker for task 4040] hadoop.MemoryManager (MemoryManager.java:updateAllocation(115)) - Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 9

2020-05-24 03:33:26,796 WARN  [Executor task launch worker for task 4122] hadoop.MemoryManager (MemoryManager.java:updateAllocation(115)) - Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
2020-05-24 03:33:27,433 WARN  [Executor task launch worker for task 4128] hadoop.MemoryManager (MemoryManager.java:updateAllocation(115)) - Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
2020-05-24 03:33:27,590 WARN  [Executor task launch worker for task 4112] hadoop.MemoryManager (MemoryManager.java:updateAllocation(115)) - Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
2020-05-24 03:33:27,702 WARN  [Executor task launch worker for task 4131] hadoop.MemoryManager (MemoryManager.java:updateAllocation(115)) - Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 8